<a href="https://colab.research.google.com/github/Shaandili/robotics-me639-2022/blob/Shaandili/ME639_Assignment_4%265_Q1%2C2%2C3%2C6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#question 1
#input = [px,py,pz]
#output = dh parameters for stanford manipulator
#d1 and a2 for stanford manipulator are 0, and the following formulae have been written accordingly
#also, only the first solution has been taken into account
import math as m
p = list(map(float,input("end effector coordinates: ").strip().split()))
theta1 = m.atan(p[1]/p[0])
r = m.sqrt(p[0]**2 + p[1]**2)
s = p[2]
theta2 = m.atan(s/r) 
d3 = m.sqrt(r**2 + s**2)

#for testing: substituting above values into dh parameter matrix for stanford manipulator, then using the code given below for calculating
#transformation matrices and manipulator jacobian

In [ ]:
#question 3
#the entire process for calculating the jacobian
#inputs = no. of links, dh parameters, revolute or prismatic?
#output = jacobian
import math as m
import numpy as np
links = int(input()) #no. of links (if spherical - include as 3 revolute joints)
dh = np.zeros((links,4)) #matrix with denavit hartenburg parameters
print("enter dh parameters for each link in the order d,theta,r,alpha")
for i in range(links-1):
  a = list(map(float,input().strip().split()))[:4]
  dh[i] = a
#calculation of transformation matrices:
H = [0]*(links) #array with link to link transformation matrices
H[0] = np.identity(4)
H0 = np.identity(4) #overall transformation matrix
for i in range(links-1):
  d = dh[i][0]
  theta = dh[i][1]
  r = dh[i][2]
  alpha = dh[i][3]
  Z = [[m.cos(theta), -1*m.sin(theta), 0, 0], #transforation matrix, Z operations
       [m.sin(theta), m.cos(theta), 0, 0],
       [0, 0, 1, d],
       [0, 0, 0, 1]]
  X = [[1, 0, 0, r], #transforation matrix, X operations
       [0, m.cos(alpha), -1*m.sin(alpha), 0],
       [0, m.sin(alpha), m.cos(alpha), 0],
       [0, 0, 0, 1]]
  H[i+1] = np.matmul(Z,X) #adding current link to link transformation to H matrix
  H0 = H0@H[i+1] #mutliplying link to link transformations to get overall transformation
on0 = [H0[0][3],H0[1][3],H0[2][3]] #position of last origin (end effector) wrt base
o = np.zeros(3) #for storing position of end effector wrt current frame
J = np.zeros((6,links)) #jacobian (initialized)
rorp = input("linkwise, enter type of joint (include end effector too): ").strip().split() #revolute or prismatic joint?
h = np.identity(4) #matrix representing transformation from base to current frame
for i in range(links):
  h = h@H[i]
  z = [h[0][2], h[1][2], h[2][2]] #z -> last column of rotation
  if rorp[i] == "prismatic": #assigning of values to jacobian column in case of prismatic joint
    J[0][i] = z[0]
    J[1][i] = z[1]
    J[2][i] = z[2]
    J[3][i] = 0
    J[4][i] = 0
    J[5][i] = 0
  else: #assigning of values to jacobian column in case of revolute joint
    oi0 = [h[0][3], h[1][3], h[2][3]] #position of current frame origin wrt base origin
    #calculation of o:
    o[0] = on0[0] - oi0[0] 
    o[1] = on0[1] - oi0[1]
    o[2] = on0[2] - oi0[2]
    zxoi = (z[1]*o[2]) - (o[1]*z[2]) 
    zxoj = (o[0]*z[2]) - (z[0]*o[2])
    zxok = (z[0]*o[1]) - (o[0]*z[1])
    zxo = [zxoi, zxoj, zxok] #cross product of z and o
    J[0][i] = zxo[0]
    J[1][i] = zxo[1]
    J[2][i] = zxo[2]
    J[3][i] = z[0]
    J[4][i] = z[1]
    J[5][i] = z[2]

print(J) #complete manipulator jacobian

In [ ]:
# rate of change of cartesian variables aka end effector velocities
xdot = list(map(float,input("xdot values: ").strip().split()))

In [ ]:
#finding pseudoinverse of jacobian:
Jtrans = np.transpose(J)
Jcross = J@Jtrans
Jcross = np.linalg.inv(Jcross)
Jcross = Jtrans@Jcross

In [ ]:
qdot = Jcross@xdot
print(qdot) #joint velocities - rate of change of each joint variable

In [ ]:
#question 2
#input = [px,py,pz], l1, l2, and d4 (the lengths of the robot links), and final transformation matrix H (0-4) (or, just H(0-4) as it contains [px,py,pz])
#taking H values from code for jacobian given above (when input is for a scara manipulator specifically)
#output = joint var for scara manipulator
import math as m
#taking in values for link lengths
l1 = float(input()) 
l2 = float(input())
d4 = float(input())
#calculation of joint variables
alpha = m.atan(H[0][0]/H[0][1]) #theta1 + theta2 - theta4 = alpha
r = m.sqrt((H[0][3]**2 + H[1][3]**2 - l1**2 - l2**2)/(2*l1*l2))
theta2 = m.atan(r/m.sqrt(1-r**2)) #only picking the first solution
theta1 = m.atan(H[1][3]/H[0][3]) - m.atan(l2*m.sin(theta2)/(l1 + l2*m.cos(theta2)))
theta4 = theta1 + theta2 - alpha
d3 = H[2][3] + d4

#for testing: substituting above values into dh parameter matrix for scara manipulator, then using the code given above for calculating
#transformation matrices and manipulator jacobian

In [ ]:
#question 6
#for spherical manipulator - it's about finding the euler angles phi, theta, psi
#just taking the first solution where theta = Atan(u33,sqrt(1-u33**2)), not theta = Atan(u33,-1*sqrt(1-u33**2))
#given transformation matrix U = np.transpose(R(0 -> 3))@R or U = H[-3]@H[-2]@H[-1]
U = H[-2]@H[-1]
U = H[-3]@U
theta5 = m.atan(m.sqrt(1 - U[2][2]**2)/U[2][2]) #theta
theta4 = m.atan(U[1][2]/U[0][2]) #phi
theta6 = m.atan(U[2][1],(-1*U[2][1])) #psi